In [76]:
from azureml.core import Environment, Workspace
from azureml.core.conda_dependencies import CondaDependencies
import sklearn

In [77]:
import pandas as pd

preproc_train=pd.read_csv('../data/train/preproc.csv')
preproc_train.drop(columns='Unnamed: 0',inplace=True)
preproc_train.columns
X = preproc_train.drop("Survived",axis=1)
y = preproc_train["Survived"]

In [78]:
ws= Workspace.from_config()

In [79]:
from azureml.core.model import Model as AMLModel
def get_model(
    model_name: str,
    model_version: int = None,  # If none, return latest model
    tag_name: str = None,
    tag_value: str = None,
    aml_workspace: Workspace = None
) -> AMLModel:
    """
    Retrieves and returns a model from the workspace by its name
    and (optional) tag.

    Parameters:
    aml_workspace (Workspace): aml.core Workspace that the model lives.
    model_name (str): name of the model we are looking for
    (optional) model_version (str): model version. Latest if not provided.
    (optional) tag (str): the tag value & name the model was registered under.

    Return:
    A single aml model from the workspace that matches the name and tag, or
    None.
    """
    if aml_workspace is None:
        print("No workspace defined - using current experiment workspace.")
        raise ValueError(
                "No workspace provided"
            )

    tags = None
    if tag_name is not None or tag_value is not None:
        # Both a name and value must be specified to use tags.
        if tag_name is None or tag_value is None:
            raise ValueError(
                "model_tag_name and model_tag_value should both be supplied"
                + "or excluded"  # NOQA: E501
            )
        tags = [[tag_name, tag_value]]

    model = None
    if model_version is not None:
        # TODO(tcare): Finding a specific version currently expects exceptions
        # to propagate in the case we can't find the model. This call may
        # result in a WebserviceException that may or may not be due to the
        # model not existing.
        model = AMLModel(
            aml_workspace,
            name=model_name,
            version=model_version,
            tags=tags)
    else:
        models = AMLModel.list(
            aml_workspace, name=model_name, tags=tags, latest=True)
        if len(models) == 1:
            model = models[0]
        elif len(models) > 1:
            raise Exception("Expected only one model")

    return model

In [80]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client= MLClient.from_config(DefaultAzureCredential())

Found the config file in: /config.json


In [86]:
model=get_model("titanic-xgb.pkl",aml_workspace=ws)

In [82]:



environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pip==20.2.4'],
    pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    
    'xgboost',
    'scikit-learn=={}'.format(sklearn.__version__)
])

In [83]:
model

Model(workspace=Workspace.create(name='mlops-aml-ws', subscription_id='f90533aa-280d-40b9-9949-a7ba0ee9511f', resource_group='mlops-rg'), name=titanic-xgb.pkl, id=titanic-xgb.pkl:5, version=5, tags={'area': 'Titanic_classification', 'run_id': '1f9fdd03-05d1-4508-92f9-237fea39c39d', 'experiment_name': 'evaluation', 'azureml.nodeid': '3d62ba08', 'azureml.pipeline': '1f9fdd03-05d1-4508-92f9-237fea39c39d', 'mlflow.source.type': 'JOB', 'mlflow.source.name': 'train.py', 'dataset_id': '84aecbe0-f2bb-4815-b21c-21bb22ffec6d', 'mlflow.parentRunId': '1f9fdd03-05d1-4508-92f9-237fea39c39d', 'mlflow.rootRunId': '1f9fdd03-05d1-4508-92f9-237fea39c39d', 'mlflow.runName': 'magenta_beach_wm03kcg4', 'mlflow.user': 'Giosuè Cotugno', 'mse': '0.02242152466367713'}, properties={})

In [84]:
#import os
#os.chdir('Users/giosue.cotugno/mlops_titanic/notebooks/')

# Model deploying

In [87]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model


service_name = 'titanic1endpoint'

inference_config = InferenceConfig(entry_script='../src/scoring/score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_13745/1012544289.py:11: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-15 16:26:37+00:00 Creating Container Registry if not exists.
2022-12-15 16:26:37+00:00 Registering the environment.
2022-12-15 16:26:38+00:00 Use the existing image.
2022-12-15 16:26:38+00:00 Generating deployment configuration.
2022-12-15 16:26:39+00:00 Submitting deployment to compute.
2022-12-15 16:26:44+00:00 Checking the status of deployment titanic1endpoint..
2022-12-15 16:29:51+00:00 Checking the status of inference endpoint titanic1endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"


AttributeError: 'NoneType' object has no attribute 'get_logs'

In [ ]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-15 14:53:34+00:00 Creating Container Registry if not exists.
2022-12-15 14:53:35+00:00 Registering the environment.
2022-12-15 14:54:07+00:00 Use the existing image.
2022-12-15 14:54:07+00:00 Generating deployment configuration.
2022-12-15 14:54:09+00:00 Submitting deployment to compute.
2022-12-15 14:54:13+00:00 Checking the status of deployment titanic1endpoint.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 4bd3b0a3-5703-4845-ae67-3de6e45f793b
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: titanic1endpoint. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image cotuamlcr.azurecr.io/azureml/azureml_be191dc2da7d62cdf3db9b0fe4fc6cff locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more informati

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 4bd3b0a3-5703-4845-ae67-3de6e45f793b
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: titanic1endpoint. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image cotuamlcr.azurecr.io/azureml/azureml_be191dc2da7d62cdf3db9b0fe4fc6cff locally. Please refer to https://aka.m

In [ ]:
service

AciWebservice(workspace=Workspace.create(name='mlops-aml-ws', subscription_id='f90533aa-280d-40b9-9949-a7ba0ee9511f', resource_group='mlops-rg'), name=titanic1endpoint, image_id=None, image_digest=None, compute_type=ACI, state=Failed, scoring_uri=None, tags=None, properties={'azureml.git.repository_uri': 'git@github.com:cotus997/mlops-titanic.git', 'mlflow.source.git.repoURL': 'git@github.com:cotus997/mlops-titanic.git', 'azureml.git.branch': 'main', 'mlflow.source.git.branch': 'main', 'azureml.git.commit': 'c2e076d2a23cea4735b3947b6066024bdd239d57', 'mlflow.source.git.commit': 'c2e076d2a23cea4735b3947b6066024bdd239d57', 'azureml.git.dirty': 'True'}, created_by={'userObjectId': '0ede9b9c-2b54-4172-a26d-bbeff841eda9', 'userPuId': '10037FFEA30E4C08', 'userIdp': None, 'userAltSecId': None, 'userIss': 'https://sts.windows.net/e99647dc-1b08-454a-bf8c-699181b389ab/', 'userTenantId': 'e99647dc-1b08-454a-bf8c-699181b389ab', 'userName': 'Giosuè Cotugno', 'upn': 'giosue.cotugno@studio.unibo.it'}

In [ ]:
import json
import numpy

input_payload = json.dumps({
    'data': X[0:2].tolist(),
    'method': 'predict_proba'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

In [ ]:
service.delete()

WebserviceException: WebserviceException:
	Message: There is a deployment operation in flight for the Service: titanic1endpoint
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "There is a deployment operation in flight for the Service: titanic1endpoint"
    }
}

# Multi model endpoint

In [ ]:
model_1=get_model("titanic-xgb.pkl",aml_workspace=ws)
model_2=get_model("my-sklearn-model",aml_workspace=ws)

In [ ]:
inference_config = InferenceConfig(entry_script="../src/scoring/score.py", environment=environment)

In [ ]:
from azureml.core.webservice import AciWebservice

aci_service_name = "aciservice-multimodel"

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(ws, aci_service_name, [model_1, model_2], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)

print(service.state)

# Test web service

In [ ]:
import json

test_sample = json.dumps({'data': X[0:2].tolist()})

prediction = service.run(test_sample)

print(prediction)

In [ ]:
service.delete()